# Real Time Adjustment

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import ast
import csv

%load_ext autoreload
%autoreload 2

from constants import *
from real_time_adj_original import *
from evaluation import *
from constants import *

## Load Data

In [70]:
def build_dataset(start_date='01-01-2022', end_date='12-30-2022'):
    prices = pd.read_csv('./data/2020_data.csv', index_col=0)[HOURS_PER_MONTH:]
    prices.index = pd.date_range(start='01-31-2021', periods=len(prices), freq='h')
    prices = prices[start_date:end_date]
    prices = prices.drop(columns=['production_FC','Offshore DK1', 'Offshore DK2', 'Onshore DK1', 'Onshore DK2'])
    prices['production_RE'] *= NOMINAL_WIND
    forecasts = pd.read_csv('./forecasting/rolling_forecasts_2021_2022.csv', index_col=0)
    forecasts.index = pd.date_range(start='01-31-2021', periods=len(forecasts), freq='h')
    forecasts = forecasts[start_date:end_date]
    forecasts *= NOMINAL_WIND
    forecasts = forecasts.clip(0)
    # For each row, create a list that contains the values of the 24 forecast columns, and add it to a new column
    forecasts['production_FC'] = forecasts.apply(lambda row: [row['FC_{i}h'.format(i=i)] for i in range(1, 25)], axis=1)
    forecasts = forecasts.drop(columns=[f'FC_{i}h' for i in range(1, 25)])
    dataset = pd.concat([prices, forecasts], axis=1)
    return dataset

data = build_dataset()
display(data)

In [71]:
def import_data(start_date='01-01-2022', end_date='12-30-2022'):
    data = build_dataset(start_date, end_date)
    prices_B = np.maximum(data["UP"].to_numpy(), 0)
    prices_S = np.maximum(data["DW"].to_numpy(), 0)
    prices_F = np.maximum(data["forward_RE"].to_numpy(), 0)
    prices_forecast = np.maximum(data["forward_FC"].to_numpy(), 0)
    realized = data.loc[:, "production_RE"].to_numpy()
    # Make a column that is a list of the 24h forecasts
    power_forecasts = data.loc[:, "production_FC"].to_numpy()
    penalty = np.quantile(prices_B, 0.95)
    
    return (prices_B, prices_S, prices_F, prices_forecast, realized, power_forecasts, penalty)

(prices_B, prices_S, prices_F, prices_forecast, realized, power_forecasts, penalty) = import_data()

display(prices_B, prices_S, prices_F, prices_forecast, realized, power_forecasts, penalty)

In [72]:
import gurobipy as gp
from gurobipy import GRB

## Hindsight 

In [166]:
df_hindsight = pd.read_csv("./results/2020/Hindsight.csv")
display(df_hindsight)

result_hindsight = test_fixed(df_hindsight['forward bid'], df_hindsight['hydrogen production'], HOURS_PER_YEAR, HOURS_PER_YEAR + 1*HOURS_PER_DAY)
print(f"Hindsight: {np.sum(result_hindsight['obj'])}")

## HAPD-AF-12

In [151]:
df_hapd_af_12 = pd.read_csv("./results/2020/HAPD-AF-12.csv")

result_hapd_af_12 = test_price_domain(df_hapd_af_12, HOURS_PER_YEAR, HOURS_PER_YEAR + 5*HOURS_PER_DAY)
print(f"HAPD-AF-12: {np.sum(result_hapd_af_12['obj'])}")
print(result_hapd_af_12['h_prod'])

In [152]:
def objective(t, hours_left, p_adj, up, dw, forward_bids):
    # t is the current time
    (prices_B, prices_S, prices_F, prices_forecast, realized, power_forecasts, penalty) = import_data()

    obj = gp.LinExpr()
    for i, j in enumerate(range(t, t + hours_left)):
        h_adj = p_adj[i]
        obj += (
                forward_bids[j] * prices_F[j]
                + PRICE_H * h_adj
                + dw[i] * prices_S[j]
                - up[i] * prices_B[j]
        )
    return obj

In [153]:
def MPC_adjustment(results_to_copy, idx_start, idx_end, printing=False):
    results = copy.deepcopy(results_to_copy)
    
    (prices_B, prices_S, prices_F, prices_forecast, realized, power_forecasts, penalty) = import_data()
    
    print("realized: ", realized)
    print("forward_bids: ", results['forward_bid'])
    print("prices_F: ", prices_F)
    print("prices_B: ", prices_B[:24])
    print("prices_S: ", prices_S)

    ds = []
    h_prods = []
    ups = []
    dws = []
    objs = []
    missing_productions = []
    missing_production = 0
    daily_count = 0

    forward_bids = results['forward_bid']
    
    for t in range(idx_start, idx_end):
        i = t % 24
        if i == 0:
            print(f"Day {t // 24 + 1}")
        if (i == 0) and t != idx_start:
            missing_production = np.maximum(H_MIN - daily_count, 0)
            daily_count = 0
            
        d = realized[t] - forward_bids[t]

        hours_left = 24 - i 
        m = gp.Model('Real Time Adjustment')

        # Variables
        p_adj = m.addMVar(shape=hours_left, vtype=GRB.CONTINUOUS, name='p_adj', lb=0.0, ub=P_H)
        up = m.addMVar(shape=hours_left, vtype=GRB.CONTINUOUS, name='up', lb=0.0)
        dw = m.addMVar(shape=hours_left, vtype=GRB.CONTINUOUS, name='dw', lb=0.0)
        up_aux = m.addMVar(shape=hours_left, vtype=GRB.CONTINUOUS, name='up_aux',lb=-GRB.INFINITY)
        dw_aux = m.addMVar(shape=hours_left, vtype=GRB.CONTINUOUS, name='dw_aux',lb=-GRB.INFINITY)

        # Objective
        print("t: ", t, "hours_left: ", hours_left)
        m.setObjective(objective(t,hours_left, p_adj, up, dw, forward_bids), GRB.MAXIMIZE)

        # Constraints
        m.addConstr(H_MIN <= p_adj.sum() + daily_count, 'Daily Production')

        for j in range(hours_left):
            print("i+j, realized, forward bid:", i+j, realized[t+j], forward_bids[t+j])
            m.addConstr(p_adj[j] <= P_H, f'p_adj_{j}')
            if j == 0:
                settlement = realized[t+j] - forward_bids[t+j] - p_adj[j]
            else:
                settlement = power_forecasts[t][j] - forward_bids[t+j] - p_adj[j]
            m.addConstr(up_aux[j] == -settlement, f'up_aux_{j}')
            m.addConstr(dw_aux[j] == settlement, f'dw_aux_{j}')
            m.addGenConstrMax(up[j], [0, up_aux[j]], name=f'up_{j}')
            m.addGenConstrMax(dw[j], [0, dw_aux[j]], name=f'dw_{j}')

        m.setParam('OutputFlag', 0)
        m.setParam('DualReductions', 0)
        m.optimize()
        
        if m.status != GRB.OPTIMAL:
            print(f"Optimization failed at {t}")
            print(m.status)
            print(m.computeIIS())
            break
        
        for j in range(hours_left):
            print(f"p_adj_{i}: {p_adj[j].x}")
            print(f"up_{i}: {up[j].x}")
            print(f"dw_{i}: {dw[j].x}")
        
        h_adj = p_adj[0].x
        
        h_adj = np.maximum(0, h_adj)
        h_adj = np.minimum(P_H, h_adj)
        daily_count += h_adj
        print("forward_bids: ", forward_bids[t])
        print("realized: ", realized[t])
        settlement = realized[t] - forward_bids[t] - h_adj
        print("settlement: ", settlement)
        print('forward_price: ', prices_F[t])
        print('buy_price: ', prices_B[t])
        print('sell_price: ', prices_S[t])
        print("daily_count: ", daily_count)
        print("missing_production: ", missing_production)
        print("adjustment: ", h_adj)
        up = np.maximum(-settlement, 0)
        dw = np.maximum(settlement, 0)
        obj = (
                forward_bids[t] * prices_F[t]
                + PRICE_H * h_adj
                + dw * prices_S[t]
                - up * prices_B[t]
                - missing_production * penalty
        )
        print("obj: ", obj)
    
    # Compute the objective with the adjusted production

        ds.append(d)
        h_prods.append(h_adj)
        ups.append(up)
        dws.append(dw)
        missing_productions.append(missing_production)
        missing_production = 0
        objs.append(obj)
    
    results['d'] = ds
    results['h_prod'] = h_prods
    results['up'] = ups
    results['dw'] = dws
    results['missing_production'] = missing_productions
    results['obj'] = objs
    
    return results

In [155]:
result_mpc = MPC_adjustment(result_hapd_af_12, 0, 5*HOURS_PER_DAY)

In [156]:
print(f"MPC: {np.sum(result_mpc['obj'])}")
print(result_mpc['h_prod'])
print(result_mpc['obj'])
print(result_mpc['dw'])
print(result_mpc['up'])

In [157]:
result_original = apply_up_and_dw_adj(result_hapd_af_12, HOURS_PER_YEAR, HOURS_PER_YEAR + 5*HOURS_PER_DAY,printing=True)
print(f"Original: {np.sum(result_original['obj'])}")
print(result_original['h_prod'])

In [159]:
print([result_original['h_prod'][i] - result_mpc['h_prod'][i] for i in range(5*HOURS_PER_DAY)])